# SQL
SQL is main database language and has been around for decades, it is most notable in systems like MySQL, Postgres, Microsoft SQL Server, Oracle Database, SQLite and more. The structure of SQL is very simple and is close to that of plain english. The syntax goes as follows

```sql
SELECT <column criteria>
FROM <table or table joins>
WHERE <filter criteria>
GROUP BY <aggregate group criteria>
HAVING <aggregate filter criteria>
ORDER BY <ordering criteria>
```

### The data
I have downloaded a sample SQLite database from here. SQLite is a filebased database, great for testing things and prototyping. Not suitable for production, however more than you need to learn how to use `SQL`. The database is already downloaded to the repo in the `inputs/flights.db` path.

By accessing the schema in sqlite3 shell it reveals there are three tables in this database. A schema describes the structure of the database, the tables within it, the column headers of each table, as well as their data types.
```sh
sqlite> .schema
CREATE TABLE airports (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [city] TEXT,
  [country] TEXT,
  [code] TEXT,
  [icao] TEXT,
  [latitude] TEXT,
  [longitude] TEXT,
  [altitude] TEXT,
  [offset] TEXT,
  [dst] TEXT,
  [timezone] TEXT
);
CREATE INDEX ix_airports_index ON airports ([index]);
CREATE TABLE airlines (
[index] INTEGER,
  [id] TEXT,
  [name] TEXT,
  [alias] TEXT,
  [iata] TEXT,
  [icao] TEXT,
  [callsign] TEXT,
  [country] TEXT,
  [active] TEXT
);
CREATE INDEX ix_airlines_index ON airlines ([index]);
CREATE TABLE routes (
[index] INTEGER,
  [airline] TEXT,
  [airline_id] TEXT,
  [source] TEXT,
  [source_id] TEXT,
  [dest] TEXT,
  [dest_id] TEXT,
  [codeshare] TEXT,
  [stops] TEXT,
  [equipment] TEXT
);
```

### Imports
Both modules are native python libraries, so no need to install anything extra

In [1]:
import sqlite3
import pandas

### Connecting to the SQLite database file
Create a connection to the sqlite db file. You can access the database directly from this but I will be passing it into a pandas function to store query results in `pandas.DataFrame`s.

**Note:** If you're connecting to a production database you will have to build a URI containing the host, username, password, port and database in order to connect. Since sqlite db is file based it doesnt need a network connection.

In [2]:
connection = sqlite3.connect('../inputs/flights.db')

### Making a basic query with `SELECT`
All queries begin with `SELECT`. This query will select all contents from airports table. Using the asterix is considered bad practice and you should specify what columns you want to pull explicitly, for performance reasons.

In [3]:
query = '''
SELECT *
FROM airports
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby
5,5,6,Wewak Intl,Wewak,Papua New Guinea,WWK,AYWK,-3.583828,143.669186,19,10,U,Pacific/Port_Moresby
6,6,7,Narsarsuaq,Narssarssuaq,Greenland,UAK,BGBW,61.160517,-45.425978,112,-3,E,America/Godthab
7,7,8,Nuuk,Godthaab,Greenland,GOH,BGGH,64.190922,-51.678064,283,-3,E,America/Godthab
8,8,9,Sondre Stromfjord,Sondrestrom,Greenland,SFJ,BGSF,67.016969,-50.689325,165,-3,E,America/Godthab
9,9,10,Thule Air Base,Thule,Greenland,THU,BGTL,76.531203,-68.703161,251,-4,E,America/Thule


### Ordering query results with `ORDER BY` clause
SQL can be built in layers, to order the data I can add the `ORDER BY` clause

In [4]:
query = '''
SELECT name, city, latitude, longitude, code, altitude
FROM airports 
ORDER BY city, name
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,name,city,latitude,longitude,code,altitude
0,South Cariboo Regional Airport,108 Mile Ranch,51.4412,-121.1958,ZML,3129
1,Aaa,Aaa,73.9,3.222,None,0
2,Aachen HBF,Aachen,50.7667,6.1,None,873
3,Flugplatz Merzbrueck,Aachen,50.823194,6.186389,AAH,570
4,Aalborg,Aalborg,57.092789,9.849164,AAL,10
5,Aalen Heidenheim Elchingen,Aalen-heidenheim,48.777833,10.264667,None,1916
6,Station Aare,Aare,63.398779,13.075956,None,1240
7,Aarhus,Aarhus,56.300017,10.619008,AAR,82
8,Aasiaat,Aasiaat,68.7,-52.75,JEG,2
9,Abadan,Abadan,30.371111,48.228333,ABD,19


### Filtering with `WHERE` clause (example 1)
Simply add another layer with a `WHERE` clause. Im going to filter by city. There are only two results for "Aachen"

In [5]:
 query = '''
SELECT name, city, latitude, longitude, code, altitude
FROM airports 
WHERE city = "Aachen"
ORDER BY city, name
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,name,city,latitude,longitude,code,altitude
0,Aachen HBF,Aachen,50.7667,6.1,None,873
1,Flugplatz Merzbrueck,Aachen,50.823194,6.186389,AAH,570


### Filtering with `WHERE` clause (example 2)
Filtering by altitude is greater than 1000. Note that in the schema, the type for altitude is TEXT. Therefore you have to apply a `CAST` on the column in order to apply numerical filters such as >, <, <=, >=.

In [8]:
 query = '''
SELECT name, city, latitude, longitude, code, altitude
FROM airports 
WHERE CAST(altitude as INT) > 1000
ORDER BY city, name
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,name,city,latitude,longitude,code,altitude
0,South Cariboo Regional Airport,108 Mile Ranch,51.4412,-121.1958,ZML,3129
1,Aalen Heidenheim Elchingen,Aalen-heidenheim,48.777833,10.264667,None,1916
2,Station Aare,Aare,63.398779,13.075956,None,1240
3,Ghazvin Azadi,Abe-ali,35.952097,50.450778,None,3769
4,Abeche,Abeche,13.847,20.844333,AEH,1788
5,Aberdeen Regional Airport,Aberdeen,45.4491,-98.4218,ABR,1302
6,Abha,Abha,18.240367,42.656625,AHB,6858
7,Abilene Rgnl,Abilene,32.411319,-99.681897,ABI,1791
8,Dyess Afb,Abilene,32.420756,-99.8546,DYS,1789
9,Nnamdi Azikiwe Intl,Abuja,9.006792,7.263172,ABV,1123


### Aggregating with SUM() and `GROUP BY` clause
Aggregate functions such as SUM(), AVG(), COUNT(), can be applied in the `SELECT` clause area of the query, you will have to also add an `ORDER BY` clause at the bottom. In this case I ran `SUM(altitude)` and grouped it by `city`. Intuitively it makes no sense to run `SUM` on `altitude` but this is just to demonstrate how to use aggregations in `SQL`.

In [19]:
query = '''
SELECT city, SUM(altitude)
FROM airports
GROUP BY city
ORDER BY SUM(altitude) DESC
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,city,SUM(altitude)
0,Denver,21980
1,Johannesburg,20883
2,Latacunga,18410
3,Nairobi,17878
4,Irkutsk,16391
5,Antarctica,16000
6,Addis Ababa,15405
7,Daocheng,14472
8,Bangda,14219
9,Kangding,14042


### Filtering an aggregated result set with `HAVING`
You will have to use the `HAVING` clause to filter an aggregated result set, because `WHERE` clause only applies to row level data. In this example I modified the query above to filter by SUM of altitude greater than 15,000.

In [23]:
query = '''
SELECT city, SUM(altitude)
FROM airports
GROUP BY city
HAVING SUM(altitude) > 15000
ORDER BY SUM(altitude) DESC
'''

data = pandas.read_sql_query(query, con=connection)

data.head(10)

,city,SUM(altitude)
0,Denver,21980
1,Johannesburg,20883
2,Latacunga,18410
3,Nairobi,17878
4,Irkutsk,16391
5,Antarctica,16000
6,Addis Ababa,15405
